# Quickstart

## Installation

CredoAI Lens can be installed with pip from PyPI as follows:

`pip install credoai`

Optional dependencies can be installed as follows:

`pip install credoai[extras]`

## Overview

CredoAI Lens is the assessment framework component of the broader CredoAI suite.
It is usable as a standalone gateway to assessments or in combination
with CredoAI's Governance Platform. 

Understanding how your AI systems are operating is the most important step in intervening upon your system. From the technically compliated questions of improving a system to the business-relevant questions of whether to deploy a system - everything is fundamentally founded upon good observability. Lens strives to make assessment comprehensive, easy, and adaptable.

### Assessments

CredoAI Lens is an entrypoint to asssessments developed by CredoAI, as well as the broader ecosystem of open source assessments. Custom analytics can also be folded in easily (see the `lens customization` notebook)

AI system assessment starts with verifying standard performance metrics to an evolving set of assessments falling under the banner of *Responsible AI*. A non-exhaustive list includes

* Fairness
* Explainability
* Performance
* Robustness

These different categories of assessment differ substantially based on whether one is 
evaluating datasets or models, what kind of model (e.g., tabular, NLP, computer vision), and the use-case. As the ecosystem develops, Lens will support assessing a broader range of AI systems. Currently, we are focused on Fairness.

### Governance

While Lens is a stand alone assessment framework, it's value is increased when combined with the CredoAI Governance Platform. The platform supports multi-stakeholder `Alignment` on how to assess your AI systems (e.g., what does good look like for this system?). It also supports translating assessment results into a Risk perspective that is scalable across your organization and understandable to diverse stakeholders.



## Lens in 5 minutes

Get your data and models together! Here we have an support vector machine trained on the Iris Dataset. No train/test split needed for this tutorial!

In [1]:
from credoai.data import fetch_creditdefault
from credoai.utils.model_utils import get_gradient_boost_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

In [4]:
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

### Using Lens

Lens interacts with Credo Artifacts which wrap models and datasets and standardizes them for use by different assessments.

In [5]:
from credoai.lens import Lens, CredoModel, CredoData
from credoai.assessment.assessments import FairnessAssessment

In [6]:
credo_model = CredoModel(name='credit_default_classifier',
                         model=model)

# no sensitive feature! need to correct. Better dataset needed
credo_data = CredoData(name='UCI-credit-default',
                       X=X, 
                       y=y.astype(int),
                       sensitive_features=sensitive_feature)

In [7]:
scope = {'metrics': ['precision_score']}

In [8]:
lens = Lens(model=credo_model,
            data=credo_data,
            assessments=[FairnessAssessment()],
            scope=scope)

In [9]:
results = lens.run_assessments()

In [10]:
results['FairnessBase']['fairness']

,value,risk,acceptable,bound_lower,bound_upper
precision_score,0.000543,NaN,True,-inf,inf
